## Cargar y combinar datasets

En este bloque, se cargan los datasets desde archivos CSV, se verifica que contengan las columnas necesarias, y se realiza un diagnóstico inicial de los datos. Además, se lleva a cabo una **recodificación de los valores de sentimiento** para unificar el esquema de etiquetas antes de procesar los datos con el modelo.

### Recodificación de valores de sentimiento
Anteriormente, las etiquetas de sentimiento tenían los siguientes valores:
- **-1**: Sentimiento negativo hacia el candidato.
- **0**: Neutralidad hacia el candidato o ausencia de mención.
- **1**: Sentimiento positivo hacia el candidato.

En el nuevo esquema de etiquetas, los valores se recodifican para adaptarse al modelo de clasificación, quedando de la siguiente manera:
- **0**: Sentimiento negativo hacia el candidato.
- **1**: Neutralidad hacia el candidato o ausencia de mención.
- **2**: Sentimiento positivo hacia el candidato.

Esta recodificación es fundamental para que el modelo pueda procesar y clasificar correctamente las etiquetas de manera uniforme.


In [ ]:
import os
import pandas as pd

# Definir función para transformar etiquetas de sentimiento
def sentiment_to_label(x):
    if pd.isna(x):
        return 1
    elif x == -1:
        return 0
    elif x == 0:
        return 1
    elif x == 1:
        return 2
    else:
        print(f"Valor de sentimiento inesperado: {x}")
        return 1

# Rutas de los archivos CSV
debates_csv = "datasets/debates.csv"
election_day_csv = "datasets/election_day.csv"

# Verificar existencia de los archivos
if not os.path.isfile(debates_csv) or not os.path.isfile(election_day_csv):
    raise FileNotFoundError("Alguno de los archivos CSV no fue encontrado.")

# Cargar los datasets
debates_df = pd.read_csv(debates_csv)
election_day_df = pd.read_csv(election_day_csv)

# Verificar las columnas requeridas
required_columns = ["comentario_editado", "Xóchitl", "Claudia", "Maynez", "Ninguno"]
for df, name in zip([debates_df, election_day_df], ["debates.csv", "election_day.csv"]):
    missing_cols = set(required_columns) - set(df.columns)
    if missing_cols:
        raise ValueError(f"El dataset {name} le falta(n) las columnas: {missing_cols}")

# Seleccionar columnas relevantes
debates_df = debates_df[required_columns]
election_day_df = election_day_df[required_columns]


## Diagnóstico y limpieza de datos

Este bloque analiza los tipos de datos, los valores únicos y detecta problemas como valores NaN o inesperados. También limpia los datos, eliminando filas con valores inválidos.


In [ ]:
# Diagnosticar tipos y valores únicos
def diagnosticar_tipos_y_valores(df, dataset_name, columnas):
    for col in columnas:
        unique_values = df[col].unique()
        data_type = df[col].dtype
        print(f"Dataset '{dataset_name}', Columna '{col}':")
        print(f"  Tipo de dato: {data_type}")
        print(f"  Valores únicos: {unique_values}\n")

diagnosticar_tipos_y_valores(debates_df, "debates.csv", required_columns[1:])
diagnosticar_tipos_y_valores(election_day_df, "election_day.csv", required_columns[1:])

# Limpieza de columnas sentimentales
for col in required_columns[1:]:
    for df in [debates_df, election_day_df]:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df.dropna(subset=[col], inplace=True)
        df = df[df[col].isin([-1, 0, 1])]


## Transformación de etiquetas y combinación final

Convierte las etiquetas de sentimiento a un formato numérico para el modelo, limpia los comentarios y combina ambos datasets en uno solo.


In [ ]:
# Aplicar transformación de etiquetas
for df in [debates_df, election_day_df]:
    for col in required_columns[1:]:
        df[col] = df[col].apply(sentiment_to_label)

# Combinar datasets
combined_df = pd.concat([debates_df, election_day_df], ignore_index=True)

# Limpiar comentarios no válidos
combined_df = combined_df[combined_df['comentario_editado'].apply(lambda x: isinstance(x, str))]

## División de datos y preparación para Huggingface

En este bloque, se divide el dataset combinado en dos conjuntos: 80% para entrenamiento y 20% para validación, utilizando una división aleatoria con `train_test_split`. Posteriormente, los datos se convierten en datasets compatibles con Huggingface.

Para que los datasets sean compatibles con los modelos de Huggingface, es necesario incluir las siguientes características generadas por el tokenizador:

- **input_ids**: Secuencias de identificadores únicos que representan cada palabra o subpalabra del texto según el vocabulario del modelo preentrenado. Estas son las entradas principales al modelo.
- **attention_mask**: Una máscara binaria que indica qué tokens son válidos (1) y cuáles son relleno (`padding`) (0). Esto permite al modelo ignorar los tokens de relleno durante el entrenamiento y la inferencia.


In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Dividir datos
train_df, valid_df = train_test_split(combined_df, test_size=0.2, random_state=42)

# Convertir a datasets Huggingface
train_dataset_hf = Dataset.from_pandas(train_df)
valid_dataset_hf = Dataset.from_pandas(valid_df)


## Tokenización

En este bloque, se utiliza un tokenizador basado en BETO (BERT en español) para procesar los textos del dataset y convertirlos en una representación numérica adecuada para el modelo.

El tokenizador realiza las siguientes funciones principales:
- **Divide el texto en subpalabras** según el vocabulario del modelo BETO. Esto permite manejar palabras desconocidas descomponiéndolas en unidades más pequeñas.
- **Asigna un identificador único (`input_ids`)** a cada subpalabra basada en el vocabulario del modelo preentrenado.
- **Genera una máscara de atención (`attention_mask`)**, que indica qué tokens son válidos (1) y cuáles son tokens de relleno (0), para que el modelo los ignore.

Los principales argumentos utilizados en la función de tokenización son:
- `text`: El texto que será tokenizado.
- `truncation=True`: Indica que los textos largos se truncarán al alcanzar la longitud máxima especificada.
- `padding="max_length"`: Asegura que todos los textos tengan la misma longitud añadiendo tokens de relleno si es necesario.
- `max_length=128`: Establece la longitud máxima de los textos procesados.


In [ ]:
from transformers import AutoTokenizer

model_path = "path_to_your_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)

def tokenize_function(examples):
    return tokenizer(
        text=examples["comentario_editado"], 
        truncation=True, 
        padding="max_length", 
        max_length=128
    )

# Tokenizar datasets
train_dataset_hf = train_dataset_hf.map(tokenize_function, batched=True)
valid_dataset_hf = valid_dataset_hf.map(tokenize_function, batched=True)


## Definición del modelo

Este bloque define un modelo basado en BETO para la clasificación multi-etiqueta. El modelo está diseñado para predecir el sentimiento hacia múltiples candidatos en un comentario. La arquitectura se compone de dos componentes principales:

1. **BETO (BERT en español)**:
   - BETO es un modelo preentrenado que transforma las entradas textuales en representaciones vectoriales de alta dimensión.
   - BETO recibe como entrada las representaciones tokenizadas (`input_ids` y `attention_mask`).
   - Genera dos tipos de salidas principales:
     - `last_hidden_state`: Representación contextual de cada token del texto.
     - `pooler_output` (o representación del token `[CLS]`): Un vector de 768 dimensiones que encapsula el significado del texto completo.

2. **Capa de perceptrón simple**:
   - Toma como entrada el vector de 768 dimensiones de BETO (representación del token `[CLS]`).
   - Genera una salida de tamaño `num_labels_per_candidate * num_candidates`, donde:
     - `num_labels_per_candidate` es la cantidad de categorías por candidato (por ejemplo, 3: negativo, neutral, positivo).
     - `num_candidates` es el número total de candidatos.
   - Utiliza una capa densa (`Linear`) para calcular las probabilidades para cada etiqueta de cada candidato.

### Flujo del modelo:
1. BETO toma el texto tokenizado y genera una representación densa de tamaño fijo.
2. El perceptrón simple proyecta esta representación en un espacio donde las etiquetas de los candidatos son predichas.
3. Las predicciones son ajustadas y evaluadas mediante la función de pérdida `CrossEntropyLoss`, aplicada individualmente a cada candidato.

El modelo implementa la clasificación multi-etiqueta al reorganizar las salidas de la capa lineal en un formato adecuado para procesar cada candidato de manera independiente.


In [ ]:
from torch import nn
import torch
from transformers import AutoModel

class BetoMultiOutput(nn.Module):
    def __init__(self, model_name, num_labels_per_candidate=3, num_candidates=4):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        hidden_size = self.bert.config.hidden_size
        self.classifier = nn.Linear(hidden_size, num_labels_per_candidate * num_candidates)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]
        logits = self.classifier(pooled_output).view(-1, 4, 3)
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            losses = [loss_fct(logits[:, i, :], labels[:, i]) for i in range(4)]
            loss = torch.stack(losses).mean()
        return {"loss": loss, "logits": logits}


## Entrenamiento

Este bloque configura el `Trainer` de Huggingface para entrenar el modelo con los datasets preparados. El `Trainer` es una herramienta poderosa que facilita el entrenamiento y evaluación de modelos. 

### Desglose de argumentos del `Trainer`:

1. **Modelo (`model`)**:
   - El modelo que se va a entrenar. En este caso, es una instancia de la clase `BetoMultiOutput`.

2. **Args (`args`)**:
   - Contiene los argumentos de configuración del entrenamiento definidos en `TrainingArguments`.

### Desglose de `TrainingArguments`:
- **output_dir**: Directorio donde se guardarán los checkpoints y el modelo entrenado.
- **eval_strategy**: Frecuencia de evaluación. Valores comunes:
  - `"epoch"`: Evalúa después de cada época.
  - `"steps"`: Evalúa después de un número específico de pasos.
- **save_strategy**: Frecuencia con la que se guardan los checkpoints. Similar a `eval_strategy`.
- **num_train_epochs**: Número de épocas (iteraciones completas sobre los datos de entrenamiento).
- **per_device_train_batch_size**: Tamaño del batch (cantidad de ejemplos procesados simultáneamente) para entrenamiento.
- **per_device_eval_batch_size**: Tamaño del batch para evaluación.
- **logging_steps**: Número de pasos de entrenamiento entre registros en el log.
- **load_best_model_at_end**: Si es `True`, carga el modelo con el mejor desempeño (según una métrica) al final del entrenamiento.
- **metric_for_best_model**: Métrica utilizada para determinar el mejor modelo durante el entrenamiento.
- **logging_dir**: Directorio donde se guardan los logs del entrenamiento.

3. **Datasets (`train_dataset` y `eval_dataset`)**:
   - `train_dataset`: Dataset de entrenamiento.
   - `eval_dataset`: Dataset de validación utilizado para evaluar el modelo.

4. **Tokenizer (`tokenizer`)**:
   - Tokenizador utilizado para preprocesar los datos antes de ingresarlos al modelo.

5. **Métricas (`compute_metrics`)**:
   - Función personalizada para calcular métricas de evaluación como la exactitud, matriz de confusión, etc.


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./beto-multi",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs"
)

trainer = Trainer(
    model=BetoMultiOutput(model_name=model_path),
    args=training_args,
    train_dataset=train_dataset_hf,
    eval_dataset=valid_dataset_hf,
    tokenizer=tokenizer
)

trainer.train()

## Evaluación y guardado del modelo

Evalúa el modelo en el conjunto de validación y guarda el modelo ajustado junto con el tokenizador.


In [ ]:
eval_results = trainer.evaluate()
print("Resultados de evaluación:", eval_results)

# Guardar modelo y tokenizador
save_path = "./mi_beto_finetuned"
os.makedirs(save_path, exist_ok=True)
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)
